In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')

## Baseline: Mean Estimator

In [2]:
df = pd.read_csv('temp/data/demand_dataset.csv')
N = int(len(df)*0.7)
mean_estimator = df.iloc[:N].groupby(['dayofweek', 'hour', 'geohash'])['demand'].mean()
mean_estimator = mean_estimator.reset_index().rename(columns={'demand':'prediction'})
mean_estimator.head()

,dayofweek,hour,geohash,prediction
0,0,0,dr5qgz7,0
1,0,0,dr5qgz8,0
2,0,0,dr5qgzc,0
3,0,0,dr5qgze,0
4,0,0,dr5qgzr,0


In [3]:
df = df.merge(mean_estimator, how='left', on=['dayofweek', 'hour', 'geohash'])
df.head()

,date,dayofweek,hour,geohash,demand,lat,lon,prediction
0,1,6,0,dr5qgz7,0,40.602036,-74.010086,0.00
1,1,6,0,dr5qgz8,1,40.603409,-74.014206,0.25
2,1,6,0,dr5qgzc,0,40.604782,-74.012833,0.00
3,1,6,0,dr5qgze,0,40.603409,-74.010086,0.00
4,1,6,0,dr5qgzr,0,40.602036,-74.004593,0.00


In [5]:
print ((df.iloc[:N].demand - df.iloc[:N].prediction)**2).mean()
print ((df.iloc[N:].demand - df.iloc[N:].prediction)**2).mean()

9.69594487744
11.278034274


## Preprocessing

In [27]:
df = pd.read_csv('temp/data/demand_dataset.csv')
df.head()

,date,dayofweek,hour,geohash,demand,lat,lon
0,1,6,0,dr5qgz7,0,40.602036,-74.010086
1,1,6,0,dr5qgz8,1,40.603409,-74.014206
2,1,6,0,dr5qgzc,0,40.604782,-74.012833
3,1,6,0,dr5qgze,0,40.603409,-74.010086
4,1,6,0,dr5qgzr,0,40.602036,-74.004593


In [28]:
mean_geohash = df.groupby('geohash')['demand'].sum()
mean_geohash.name = 'isActive'
df = df.merge((mean_geohash>10).reset_index(), how='left', on='geohash')
df.head()

,date,dayofweek,hour,geohash,demand,lat,lon,isActive
0,1,6,0,dr5qgz7,0,40.602036,-74.010086,False
1,1,6,0,dr5qgz8,1,40.603409,-74.014206,False
2,1,6,0,dr5qgzc,0,40.604782,-74.012833,False
3,1,6,0,dr5qgze,0,40.603409,-74.010086,False
4,1,6,0,dr5qgzr,0,40.602036,-74.004593,False


In [29]:
df = df[df.isActive].drop(['isActive', 'geohash'], axis=1)
df.head()

,date,dayofweek,hour,demand,lat,lon
19,1,6,0,1,40.602036,-73.993607
45,1,6,0,0,40.604782,-73.971634
54,1,6,0,0,40.600662,-73.960648
71,1,6,0,0,40.600662,-73.941422
82,1,6,0,0,40.613022,-74.033432


In [30]:
def create_features(df):
    df['weekend'] = (df.dayofweek > 4)
    df['dayofweek_sin'] = np.sin(df.dayofweek/7.0)
    df['dayofweek_cos'] = np.cos(df.dayofweek/7.0)
    df['hour_sin'] = np.sin(df.hour/24.0)
    df['hour_cos'] = np.cos(df.hour/24.0)
#     df['hour_sin2'] = df.hour_sin**2
#     df['hour_cos2'] = df.hour_cos**2
    df = df.drop(['date', 'dayofweek', 'hour'], axis=1)
#     df['prev_demand'] = 0
#     df['prev_demand'].iloc[1:] = df['count'].values[:-1]
#     df = df.loc[2:]
    return df

df = create_features(df)
# df = df.iloc[:2000000]
df.head()

,demand,lat,lon,weekend,dayofweek_sin,dayofweek_cos,hour_sin,hour_cos
19,1,40.602036,-73.993607,True,0.755975,0.6546,0,1
45,0,40.604782,-73.971634,True,0.755975,0.6546,0,1
54,0,40.600662,-73.960648,True,0.755975,0.6546,0,1
71,0,40.600662,-73.941422,True,0.755975,0.6546,0,1
82,0,40.613022,-74.033432,True,0.755975,0.6546,0,1


In [31]:
X = df.drop(['demand'], axis=1).values
y = df['demand'].values
N = int(len(y)*0.7)
X_train, X_test, y_train, y_test = X[:N], X[N:], y[:N], y[N:]
print len(X_train)
print len(X_test)

5417882
2321950


In [32]:
from sklearn.preprocessing import StandardScaler

scl = StandardScaler()
X_train = scl.fit_transform(X_train)
X_test = scl.transform(X_test)

/Users/odatakuma/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


In [33]:
print y_train[y_train>0].size/float(y_train.size)
print y_test[y_test>0].size/float(y_test.size)
print (y_train**2).mean()
print (y_test**2).mean()

0.265453732658
0.236592519219
50.5026333169
35.9411145804


## Linear Regression

In [34]:
from sklearn.linear_model import LinearRegression

slr = LinearRegression()
slr.fit(X_train, y_train)
print "Rsquare train/val: %.3f / %.3f" % (slr.score(X_train, y_train), slr.score(X_test, y_test))                           
print "MSE train/val: %.1f / %.1f" % (((y_train - slr.predict(X_train))**2).mean(), ((y_test - slr.predict(X_test))**2).mean())

Rsquare train/val: 0.039 / 0.055
MSE train/val: 45.7 / 31.8


## Gradient Boosting

In [35]:
from sklearn.ensemble import GradientBoostingRegressor

In [36]:
%%time
lr = 1.0
for d in [4, 8]:
    for n in [10, 20]:
        for s in [0.2, 0.5]:
            gbr = GradientBoostingRegressor(learning_rate=lr, n_estimators=n, subsample=s, max_depth=d)
            gbr.fit(X_train, y_train)
            print "d=%d, n=%d, s=%.1f, MSE train/val: %.1f / %.1f" % (d, n, s, ((y_train - gbr.predict(X_train))**2).mean(), ((y_test - gbr.predict(X_test))**2).mean())

d=4, n=10, s=0, MSE train/val: 36.1 / 22.4
d=4, n=10, s=0, MSE train/val: 37.5 / 23.5
d=4, n=20, s=0, MSE train/val: 35.2 / 21.6
d=4, n=20, s=0, MSE train/val: 34.9 / 21.4
d=8, n=10, s=0, MSE train/val: 28.2 / 20.5
d=8, n=10, s=0, MSE train/val: 28.1 / 19.3
d=8, n=20, s=0, MSE train/val: 27.0 / 19.4
d=8, n=20, s=0, MSE train/val: 25.2 / 18.0
CPU times: user 25min 53s, sys: 24.8 s, total: 26min 18s
Wall time: 26min 18s


In [37]:
%%time
lr = 1.0
s = 1.0
d = 10
n = 20
gbr = GradientBoostingRegressor(learning_rate=lr, n_estimators=n, subsample=s, max_depth=d)
gbr.fit(X_train, y_train)
print "d=%d, n=%d, s=%.1f, MSE train/val: %.1f / %.1f" % (d, n, s, ((y_train - gbr.predict(X_train))**2).mean(), ((y_test - gbr.predict(X_test))**2).mean())

d=10, n=20, s=1.0, MSE train/val: 20.9 / 16.7
CPU times: user 25min 12s, sys: 6.22 s, total: 25min 18s
Wall time: 25min 29s


## Random Forest

In [ ]:
%%time
from sklearn.ensemble import RandomForestRegressor

forest = RandomForestRegressor(n_estimators=100,
#                                max_depth=10,
                               min_samples_split=100,
                               n_jobs=-1)
forest.fit(X_train, y_train)
print "Rsquare train/val: %.3f / %.3f" % (forest.score(X_train, y_train), forest.score(X_test, y_test))                           
print "MSE train/val: %.1f / %.1f" % (((y_train - forest.predict(X_train))**2).mean(), ((y_test - forest.predict(X_test))**2).mean())

## Neural Network

In [39]:
from keras.models import Sequential
from keras.layers.core import Dense

def MLP(size, act='relu'):
    model = Sequential()
    model.add(Dense(size, input_dim=7, activation=act))
    model.add(Dense(size, activation=act))
    model.add(Dense(size, activation=act))
    model.add(Dense(1, init='normal'))
    model.compile(loss='mean_squared_error',  optimizer='adam')
    return model

Using TensorFlow backend.


In [40]:
%%time
mlp = MLP(64)
mlp.fit(X_train, y_train, nb_epoch=10, batch_size=1000, verbose=2, validation_split=0.0)

Epoch 1/10
31s - loss: 38.4921
Epoch 2/10
31s - loss: 37.3146
Epoch 3/10
30s - loss: 36.8824
Epoch 4/10
30s - loss: 36.4521
Epoch 5/10
33s - loss: 36.1121
Epoch 6/10
33s - loss: 35.8661
Epoch 7/10
31s - loss: 35.6682
Epoch 8/10
36s - loss: 35.4801
Epoch 9/10
31s - loss: 35.3002
Epoch 10/10
33s - loss: 35.1522
CPU times: user 11min 54s, sys: 1min 33s, total: 13min 27s
Wall time: 5min 25s


In [41]:
y_pred_train = mlp.predict(X_train, batch_size=10000, verbose=0)[:, 0]
y_pred_test = mlp.predict(X_test, batch_size=10000, verbose=0)[:, 0]
                         
print "MSE train/val: %.1f / %.1f" % (((y_train - y_pred_train)**2).mean(), ((y_test - y_pred_test)**2).mean())

MSE train/val: 35.0 / 23.2
